In [52]:
import pandas as pd
import numpy as np
from glob import glob
import re
import datetime


In [53]:
dirr = "C:/Users/shic/Desktop/crawler_project/data_save/kb_land/final/"
list_files = glob(dirr+'*.csv')
list_files

['C:/Users/shic/Desktop/crawler_project/data_save/kb_land/final\\매매가격_변화율.csv',
 'C:/Users/shic/Desktop/crawler_project/data_save/kb_land/final\\매매가격_지수.csv',
 'C:/Users/shic/Desktop/crawler_project/data_save/kb_land/final\\매매시장_매매거래.csv',
 'C:/Users/shic/Desktop/crawler_project/data_save/kb_land/final\\매매시장_매수우위.csv']

In [121]:
meme_diff = pd.read_csv(list_files[0])
meme_jisu = pd.read_csv(list_files[1])

In [122]:
meme_diff['rate_d_conv'] = pd.to_datetime(meme_diff['rate_d'],format='%Y,%m,%d')
meme_jisu['jisu_d_conv'] = pd.to_datetime(meme_jisu['jisu_d'],format='%Y,%m,%d')

In [123]:
meme_jisu['jisu_d_conv'][0].isocalendar()[1]

9

In [136]:
meme_jisu['jisu_d_week'] = [k.isocalendar()[1] for k in meme_jisu['jisu_d_conv']]
meme_jisu

,year,month,region_b,region_s,ref_date,date_100,jisu_v,jisu_d,jisu_d_conv,jisu_d_week
0,2012,1,서울,서울강북,(기준일: 2012.01.30),(2019.01.14=100.0),82.2,"2011,2,28",2011-02-28,9
1,2012,1,서울,서울강북,(기준일: 2012.01.30),(2019.01.14=100.0),82.2,"2011,3,7",2011-03-07,10
2,2012,1,서울,서울강북,(기준일: 2012.01.30),(2019.01.14=100.0),82.3,"2011,3,14",2011-03-14,11
3,2012,1,서울,서울강북,(기준일: 2012.01.30),(2019.01.14=100.0),82.3,"2011,3,21",2011-03-21,12
4,2012,1,서울,서울강북,(기준일: 2012.01.30),(2019.01.14=100.0),82.3,"2011,3,28",2011-03-28,13
...,...,...,...,...,...,...,...,...,...,...
856732,2021,6,경남,통영,(기준일: 2021.06.28),(2019.01.14=100.0),100.7,"2021,5,31",2021-05-31,22
856733,2021,6,경남,통영,(기준일: 2021.06.28),(2019.01.14=100.0),100.9,"2021,6,7",2021-06-07,23
856734,2021,6,경남,통영,(기준일: 2021.06.28),(2019.01.14=100.0),100.9,"2021,6,14",2021-06-14,24
856735,2021,6,경남,통영,(기준일: 2021.06.28),(2019.01.14=100.0),101.0,"2021,6,21",2021-06-21,25


In [145]:
meme_jisu.groupby(['year','month','region_b','region_s','jisu_d_week','date_100'],as_index=False)['jisu_v'].mean()

,year,month,region_b,region_s,jisu_d_week,date_100,jisu_v
0,2012,1,강원,강릉,1,(2019.01.14=100.0),84.2
1,2012,1,강원,강릉,2,(2019.01.14=100.0),84.3
2,2012,1,강원,강릉,3,(2019.01.14=100.0),84.6
3,2012,1,강원,강릉,5,(2019.01.14=100.0),84.8
4,2012,1,강원,강릉,9,(2019.01.14=100.0),72.6
...,...,...,...,...,...,...,...
856732,2021,6,충북,충주,49,(2019.01.14=100.0),96.3
856733,2021,6,충북,충주,50,(2019.01.14=100.0),96.9
856734,2021,6,충북,충주,51,(2019.01.14=100.0),97.3
856735,2021,6,충북,충주,52,(2019.01.14=100.0),97.9
